# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


# What test is appropriate for this problem? Does CLT apply?

Since the sample size is quite large and the 'b' and 'w' values in race are assigned randomly, CLT applies. We will use a boostrapping test and a two-sample z-test and t-test for this problem. Because the sample size is quite large (>30), t-test and z-test will yield consistent results.

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']
print('sample size of white:',w.shape)
print('sample size of black:',b.shape)

sample size of white: (2435, 65)
sample size of black: (2435, 65)


# What are the null and alternate hypotheses?
The null hypothesis: there is no difference in getting callbacks between blacks and whites.<br> 
The alternative hypothesis: there is difference in getting callbacks between the two groups 'b' and 'w'.

# Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [6]:
w_call=w.call
b_call=b.call

## frequentist approach

In [7]:
w_std=w_call.std()
b_std=b_call.std()
diff_mean=np.abs(np.mean(w_call) - np.mean(b_call))
print('white std:\t',w_std,'\nblack std:\t',b_std,'\ndiff in mean:\t',diff_mean)
std_err=np.sqrt(w_std**2/len(w_call) + b_std**2/len(b_call))
print('std error:\t',std_err)

confidence_interval=diff_mean+np.array([-1,1])*1.96*std_err 
print('95% confidence:',confidence_interval)

white std:	 0.2953455150127411 
black std:	 0.24564945697784424 
diff in mean:	 0.03203285485506058
std error:	 0.007784906919813793
95% confidence: [0.01677444 0.04729127]


In [8]:
#p-value
t, p_t = stats.ttest_ind(w_call,b_call)
print('t score:\t',t,'\np val by t-test:',p_t)


z = diff_mean /std_err
p_z=stats.norm.sf(abs(z)) * 2
print('\nz score\t\t:', z,'\np value by z-test:',p_z)



t score:	 4.114705290861751 
p val by t-test: 3.940802103128886e-05

z score		: 4.114738324427747 
p value by z-test: 3.876188895481077e-05


## Bootstrapping approach

In [9]:
permutation_replicates = np.empty(10000)

for i in range(len(permutation_replicates)):
    permutation_samples = np.random.permutation(np.concatenate((w_call, b_call)))

    w_perm = permutation_samples[:len(w_call)]
    b_perm = permutation_samples[len(b_call):]

    permutation_replicates[i] = np.abs(np.mean(w_perm) - np.mean(b_perm))

p = np.sum(permutation_replicates > diff_mean) / len(permutation_replicates)
print('p =', p)

p = 0.0


Both frequentist and bootstrapping approach yields rather low p value. Therefore we reject the null hypothesis and conclude that there is significant difference in getting callbacks between the two groups 'black' and 'white'.

# Write a story describing the statistical significance in the context or the original problem.

Racial discrimination is pervasive in cultures throughout the world. We have investigated the data in the Job Market in the United States. The p-value found via t-test/z-test and permutations are well below the threshold of 0.05. Therefore we reject the null hypothesis in favor of the alternate hypothesis. We conclude that there is significant impact of race on the callback rates.

# Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Our analysis shows that race has a significant impact of the callback rates but it doesn't necessarily mean that race/name is the most important factor in callback success. We don't knkow if the callback success is associated with other features. So a further investigation on correlations with features like 'education','jobexpereice' is needed.